In [1]:
import sys
sys.path.append('./SimuladorEP')
import cap_adt as cap
import path_adt as path
import tube_adt as tube
import graph_adt as graph
import ind_adt as ind
import event_adt as event
import random
from functools import reduce
from random import random as rand
from math import log
import time

def exprandom(m):
    x=rand()
    return -m*log(x)

In [2]:
def simulador(g,o,d,k,ht,tbc,tbd,tbz):
    #inicio
    agenda=cap.newS()
    c=len(graph.get_nodesG(g))
    
    #inicializacao
    t=tube.newT()
    a=graph.get_edgesG(g)
    i=1
    for j in range(k):
        for e in a:
            (x,y,z)=(random.uniform(0,1),random.uniform(0,1),random.uniform(0,1))
            i1=ind.newI(path.
                        newP(e[0],e[1]),x,y,z,i)
            t=tube.addT(t,i1)
            agenda=cap.addS(agenda,event.newE(exprandom(tbc),'conc',i))
            agenda=cap.addS(agenda,event.newE(exprandom(tbd),'des',i))
            i=i+1
    agenda=cap.addS(agenda,event.newE(exprandom(tbz),'cis','all'))
    
    ce=cap.nextS(agenda) 
    ct=event.timeE(ce)
    ck=event.kindE(ce)
    ci=event.idE(ce)
    
    while ct<=ht:
    #eventos   
        if ck=='conc' and tube.existsT(t,ci):
            i1=tube.get_indT(t,ci)
            l=tube.neigT(t,ci)
            if len(l)!=0 and not (len(l)==1 and ind.eqI(i1,l[0])):
                i2=random.choice(list(filter(lambda x:not ind.eqI(x,i1),l)))
                if path.firstP(ind.pathI(i2))==path.lastP(ind.pathI(i1)):
                    (x,y,z)=((ind.xposI(i1)+ind.xposI(i2))/2,(ind.yposI(i1)+ind.yposI(i2))/2,(ind.zposI(i1)+ind.zposI(i2))/2)
                    p=path.glueP(ind.pathI(i1),ind.pathI(i2))
                    t=tube.addT(t,ind.newI(p,x,y,z,i))
                    t=tube.delT(t,ci)
                    t=tube.delT(t,ind.idI(i2))
                else:
                    (x,y,z)=((ind.xposI(i1)+ind.xposI(i2))/2,(ind.yposI(i1)+ind.yposI(i2))/2,(ind.zposI(i1)+ind.zposI(i2))/2)
                    p=path.glueP(ind.pathI(i2),ind.pathI(i1))
                    t=tube.addT(t,ind.newI(p,x,y,z,i))
                    t=tube.delT(t,ci)
                    t=tube.delT(t,ind.idI(i2))
                agenda=cap.addS(agenda,event.newE(ct+exprandom(tbc),'conc',i))
                agenda=cap.addS(agenda,event.newE(ct+exprandom(tbd),'des',i))
                i+=1
            else:
                agenda=cap.addS(agenda,event.newE(ct+exprandom(tbc),'conc',ci))
            
        
        elif ck=='des' and tube.existsT(t,ci):
            i1=tube.get_indT(t,ci)
            dist=1/path.lenP(ind.pathI(i1))
            r=[ind.xposI(i1),ind.yposI(i1),ind.zposI(i1)]
            v=[]
            for a in r:
                if a-dist<0:
                    v=v+[[0,a+dist]]
                elif a+dist>1:  
                    v=v+[[a-dist,1]]
                else:
                    v=v+[[a-dist,a+dist]]
            (x,y,z)=(random.uniform(v[0][0],v[0][1]),random.uniform(v[1][0],v[1][1]),random.uniform(v[2][0],v[2][1]))
            t=tube.addT(t,ind.newI(ind.pathI(i1),x,y,z,ci))
            agenda=cap.addS(agenda,event.newE(ct+exprandom(tbd),'des',ci))
            
        elif ck=='cis':
            todo=[tube.get_indT(t,m) for m in tube.get_idsT(t)]
            while todo!=[]:
                i1=todo[0]
                comp=path.lenP(ind.pathI(i1))
                if ind.idI(i1)==-1 or comp>c:
                    x=random.choice(range(1,(comp-1))) 
                    (a,b)=path.breakP(ind.pathI(i1),x)
                    if path.lenP(a)>c//2:
                        todo+=[ind.newI(a,0,0,0,-1)]
                    else:
                        (x,y,z)=(random.uniform(0,1),random.uniform(0,1),random.uniform(0,1))
                        t=tube.addT(t,ind.newI(a,x,y,z,i))
                        agenda=cap.addS(agenda,event.newE(ct+exprandom(tbc),'conc',i))
                        agenda=cap.addS(agenda,event.newE(ct+exprandom(tbd),'des',i))
                        i+=1
                    if path.lenP(b)>c//2:
                        todo+=[ind.newI(b,0,0,0,-1)]
                    else:
                        (x,y,z)=(random.uniform(0,1),random.uniform(0,1),random.uniform(0,1))
                        t=tube.addT(t,ind.newI(b,x,y,z,i))
                        agenda=cap.addS(agenda,event.newE(ct+exprandom(tbc),'conc',i))
                        agenda=cap.addS(agenda,event.newE(ct+exprandom(tbd),'des',i))
                        i+=1
                    if ind.idI(i1)!=-1:
                        t=tube.delT(t,ind.idI(i1))
                todo=todo[1:]
            agenda=cap.addS(agenda,event.newE(ct+exprandom(tbz),'cis','all'))
            
        #Atualização da agenda e das variáveis após ocorrência de um evento
        ce=cap.nextS(agenda)
        ct=event.timeE(ce)
        ck=event.kindE(ce)
        ci=event.idE(ce)
        agenda=cap.delS(agenda)

#filtragem
    r=[]
    for m in tube.get_idsT(t):
        r=r+[ind.pathI(tube.get_indT(t,m))]
    res=[]
    for p in r:
        ok=False
        if p not in res:
            ok=True
            if path.firstP(p)!=o or path.lastP(p)!=d:
                ok=False
            elif path.lenP(p)!=c:
                ok=False
            else:
                lista=graph.get_nodesG(g)
                while ok and lista!=[]: 
                    if not(path.crossesP(p,lista[0])):
                        ok=False
                    lista=lista[1:]
        if ok:
            res=res+[p]
    return res if res!=[] else 'Não foram encontrados caminhos Hamiltonianos'

 

In [33]:
#Example
G = graph.newG([1, 2, 3, 4])
G = graph.add_edgesG(G, [(1, 2), (1, 3), (2, 3), (3, 2), (3, 4), (2, 4)])

simulador(G, 1, 4, 20, 20, 1, 1, 1)

[[1, 2, 3, 4], [1, 3, 2, 4]]